In [3]:
import os
import pymysql
from dotenv import load_dotenv
load_dotenv()

HOST = os.getenv('HOST')
USER = os.getenv('MYSQL_USER')
PASSWORD = os.getenv('MYSQL_ROOT_PASSWORD')
DB = os.getenv('MYSQL_DATABASE')

try:
    connection = pymysql.connect(host=HOST, 
                          user = USER,
                         password = PASSWORD,
                         db = DB)

    cursor = connection.cursor()
    print("Connection stablished.")
except:
    print("An exception occurred!")

Connection stablished.


## Attribute Selection
* __Dropping unecessary columns__

In [51]:
cursor.execute("SELECT * FROM pedidos")
cursor.fetchall()[0]

"""
Caption:
numero
data_pedido
hora_pedido
cliente
endereco
telefone
tipo_entrega
valor_pizza
valor_borda
valor_refrigerante
valor_entrga
valor_total
hora_entrega
tempo
"""

(1,
 '2011-10-07',
 '18:45:00',
 'Cliente 1',
 'Endereco 1',
 '00-000-0000',
 'Buscar',
 32.0,
 0.0,
 0.0,
 0.0,
 32.0,
 '19:15:00',
 '00:30:00')

In [2]:
#Defining the columns that are going to be droped
numero = "ALTER TABLE pedidos DROP COLUMN numero;"
cliente = "ALTER TABLE pedidos DROP COLUMN cliente"
endereco = "ALTER TABLE pedidos DROP COLUMN endereco"
telefone = "ALTER TABLE pedidos DROP COLUMN telefone"
valor_pizza = "ALTER TABLE pedidos DROP COLUMN valor_pizza"
valor_entrega = "ALTER TABLE pedidos DROP COLUMN valor_entrega"
hora_entrega = "ALTER TABLE pedidos DROP COLUMN hora_entrega"

In [3]:
lista = [numero, cliente, endereco, telefone,
        valor_pizza, valor_entrega, 
        hora_entrega]

In [4]:
for i in range(len(lista)):
    cursor.execute(lista[i])
connection.commit()

In [7]:
cursor.execute("SELECT dayname(data_pedido) FROM pedidos")
cursor.fetchall()[0:10]

(('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',))

In [12]:
cursor.execute("SELECT max(data_pedido), min(data_pedido) FROM pedidos")
cursor.fetchone()

(datetime.date(2015, 2, 3), datetime.date(2011, 10, 7))

>__As from this database the entities with data_peido >= 2015 are only a test without value, I am going to remove them__

In [13]:
delete_data = "DELETE FROM pedidos WHERE year(data_pedido) >= 2015"
cursor.execute(delete_data)
connection.commit()

In [15]:
cursor.execute("SELECT max(data_pedido), min(data_pedido) FROM pedidos")
cursor.fetchone()

(datetime.date(2012, 2, 24), datetime.date(2011, 10, 7))

In [2]:
cursor.execute("SELECT data_pedido, dayname(data_pedido) FROM pedidos")
cursor.fetchall()[0:5]

((datetime.date(2011, 10, 7), 'Friday'),
 (datetime.date(2011, 10, 7), 'Friday'),
 (datetime.date(2011, 10, 7), 'Friday'),
 (datetime.date(2011, 10, 7), 'Friday'),
 (datetime.date(2011, 10, 7), 'Friday'))

In [4]:
#Returning time and dayname
cursor.execute("SELECT DATE_FORMAT(hora_pedido, '%H:%i'), dayname(hora_pedido) FROM pedidos")
cursor.fetchall()[0:5]

(('18:45', 'Tuesday'),
 ('18:47', 'Tuesday'),
 ('18:49', 'Tuesday'),
 ('18:50', 'Tuesday'),
 ('18:52', 'Tuesday'))

## Creating Dataframe from Sqlite
>Is a good practice to do this way, because I don't need to change the sql, and dataframe provides more features.

>Another question is because I need to define sales period from my database.

In [35]:
import sqlite3
import pandas as pd
connection_sqlite = sqlite3.connect('source/pizzaria.db')

df = pd.read_sql_query("SELECT * FROM pedidos", connection_sqlite)
df

,numero,data_pedido,hora_pedido,cliente,endereco,telefone,tipo_entrega,valor_pizza,valor_borda,valor_refrigerante,valor_entrega,valor_total,hora_entrega,tempo
0,1,2011-10-07,18:45:00,Cliente 1,Endereco 1,00-000-0000,Buscar,32.0,0.0,0.0,0.0,32.0,19:15:00,00:30:00
1,2,2011-10-07,18:47:00,Cliente 2,Endereco 2,00-000-0001,Buscar,30.0,0.0,0.0,0.0,30.0,19:30:00,00:43:00
2,3,2011-10-07,18:49:00,Cliente 3,Endereco 3,00-000-0002,Buscar,27.0,0.0,0.0,0.0,27.0,19:31:00,00:42:00
3,4,2011-10-07,18:50:00,Cliente 4,Endereco 4,00-000-0003,Buscar,30.0,0.0,0.0,0.0,30.0,19:30:00,00:40:00
4,5,2011-10-07,18:52:00,Cliente 5,Endereco 5,00-000-0004,Buscar,30.0,0.0,0.0,0.0,30.0,19:15:00,00:23:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2267,2268,2015-02-03,22:00:00,Cliente 2268,Endereco 2268,00-000-2267,Entrega,28.0,0.0,4.5,2.5,35.0,22:40:00,00:40:00
2268,2269,2015-02-03,22:17:00,Cliente 2269,Endereco 2269,00-000-2268,Buscar,22.0,0.0,0.0,0.0,22.0,23:00:00,00:43:00
2269,2270,2015-02-03,22:44:00,Cliente 2270,Endereco 2270,00-000-2269,Entrega,20.0,0.0,0.0,2.5,22.5,23:30:00,00:46:00
2270,2271,2015-02-03,22:54:00,Cliente 2271,Endereco 2271,00-000-2270,Entrega,27.0,0.0,0.0,2.5,27.0,23:24:00,00:30:00


## Getting the day name from data_pedido

In [40]:
day_name= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
week_day = []
for i in df['data_pedido']:
    week_day.append(day_name[datetime.datetime.strptime(i,'%Y-%m-%d').weekday()])

In [42]:
df['dia_da_semana'] = week_day

In [43]:
df

,numero,data_pedido,hora_pedido,cliente,endereco,telefone,tipo_entrega,valor_pizza,valor_borda,valor_refrigerante,valor_entrega,valor_total,hora_entrega,tempo,dia_da_semana
0,1,2011-10-07,18:45:00,Cliente 1,Endereco 1,00-000-0000,Buscar,32.0,0.0,0.0,0.0,32.0,19:15:00,00:30:00,Friday
1,2,2011-10-07,18:47:00,Cliente 2,Endereco 2,00-000-0001,Buscar,30.0,0.0,0.0,0.0,30.0,19:30:00,00:43:00,Friday
2,3,2011-10-07,18:49:00,Cliente 3,Endereco 3,00-000-0002,Buscar,27.0,0.0,0.0,0.0,27.0,19:31:00,00:42:00,Friday
3,4,2011-10-07,18:50:00,Cliente 4,Endereco 4,00-000-0003,Buscar,30.0,0.0,0.0,0.0,30.0,19:30:00,00:40:00,Friday
4,5,2011-10-07,18:52:00,Cliente 5,Endereco 5,00-000-0004,Buscar,30.0,0.0,0.0,0.0,30.0,19:15:00,00:23:00,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2267,2268,2015-02-03,22:00:00,Cliente 2268,Endereco 2268,00-000-2267,Entrega,28.0,0.0,4.5,2.5,35.0,22:40:00,00:40:00,Tuesday
2268,2269,2015-02-03,22:17:00,Cliente 2269,Endereco 2269,00-000-2268,Buscar,22.0,0.0,0.0,0.0,22.0,23:00:00,00:43:00,Tuesday
2269,2270,2015-02-03,22:44:00,Cliente 2270,Endereco 2270,00-000-2269,Entrega,20.0,0.0,0.0,2.5,22.5,23:30:00,00:46:00,Tuesday
2270,2271,2015-02-03,22:54:00,Cliente 2271,Endereco 2271,00-000-2270,Entrega,27.0,0.0,0.0,2.5,27.0,23:24:00,00:30:00,Tuesday
